In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

# подготовка данных

In [ ]:
# тут мы подгружаем датасет
import pandas as pd
from datasets import Dataset

df_0k1k = pd.read_csv('cases_random_0kto1k.csv')
df_1k2k = pd.read_csv('cases_random_1kto2k.csv')
df_2k3k = pd.read_csv('cases_random_2kto3k.csv')
df_3k4k = pd.read_csv('cases_random_3kto4k.csv')
df_4k5k = pd.read_csv('cases_random_4kto5k.csv')
df_5k6k = pd.read_csv('cases_random_5kto6k.csv')
df_6k7k = pd.read_csv('cases_random_6kto7k.csv')
df_7k8k = pd.read_csv('cases_random_7kto8k.csv')
df_8k9k = pd.read_csv('cases_random_8kto9k.csv')
df_9k10k = pd.read_csv('cases_random_9kto10k.csv')

In [ ]:
# df_50 = pd.read_csv('example50.csv')  # сейчас тут 50 строк, нужно добавить тексты приговоров до 150
df_150 = pd.read_csv('example150.csv')
# df_100 = pd.read_csv('example100.csv')

In [ ]:
# тут мы выделяем последнюю часть текста, в которой содержится информация о статьях и наказании
import re

def extract_after_prigovoril(text):
    import re
    if not isinstance(text, str):
        return ""
    match = re.search(r"(приговорил|п\s*р\s*и\s*г\s*о\s*в\s*о\s*р\s*и\s*л)\s*[:]", text.lower())
    if match:
        return text[match.end():].strip()
    else:
        return text  # если нет "приговорил", брать весь текст

# подключение и обучение модели

In [ ]:
# это чтобы не подключаться к WabdB и чтобы он не беспокоил просьбами об апи ключе
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# подключаемся к большой модели, обученной на русском языке, с пайпланом классификации текстов
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=6)

# Метки
label2id = {
    "штраф": 0,
    "условное осуждение": 1,
    "лишение свободы": 2,
    "ограничение свободы": 3,
    "обязательные работы": 4,
    "исправительные работы": 5,
    "принудительные работы": 6
}
id2label = {v: k for k, v in label2id.items()}

def preprocess(example):
    text = extract_after_prigovoril(example['result_text'])
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    encoding['labels'] = label2id[example['result_type']]
    return encoding

# берем из размеченного напи датасета только текст приговора и нужную сейчас характеристику (result_type)
train_dataset = Dataset.from_pandas(df_150[['result_text', 'result_type']])
train_dataset = train_dataset.map(preprocess, batched=False)

# Настройка обучения
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


TrainOutput(global_step=95, training_loss=0.7707706652189555, metrics={'train_runtime': 85.4034, 'train_samples_per_second': 8.723, 'train_steps_per_second': 1.112, 'total_flos': 196024776099840.0, 'train_loss': 0.7707706652189555, 'epoch': 5.0})

In [ ]:
# сохраняем модель и токенизатор, чтобы потом снова не обучать (ну, придется обучить еще раз на датасете из 150 кейсов)
model.save_pretrained("./my_model")
tokenizer.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json')

# загрузка нашей обученной модели и предсказание значений

In [ ]:
# загружаем нашу обученную на примере модель
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("./my_model")
tokenizer = BertTokenizer.from_pretrained("./my_model")
model.eval()  # переводим в режим инференса

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# предсказываем значение характеристики для неразмеченного большого датасета
from torch import no_grad

def predict_result_type(text):
    text = extract_after_prigovoril(text)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=512)
    with no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax().item()
    return id2label[predicted_class_id]



In [ ]:
# 1k
big_df = df_0k1k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_0k1k.csv", index=False)

predicted_result_type_df_0k1k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_0k1k.to_csv("predicted_result_type_0k1k.csv", index=False)

In [ ]:
# 1k to 2k
big_df = df_1k2k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_1k2k.csv", index=False)

predicted_result_type_df_1k2k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_1k2k.to_csv("predicted_result_type_1k2k.csv", index=False)

In [ ]:
# 2k to 3k
big_df = df_2k3k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_2k3k.csv", index=False)

predicted_result_type_df_2k3k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_2k3k.to_csv("predicted_result_type_2k3k.csv", index=False)

In [ ]:
# 3k to 4k
big_df = df_3k4k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_3k4k.csv", index=False)

predicted_result_type_df_3k4k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_3k4k.to_csv("predicted_result_type_3k4k.csv", index=False)

In [ ]:
# 4k to 5k
big_df = df_4k5k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_4k5k.csv", index=False)

predicted_result_type_df_4k5k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_4k5k.to_csv("predicted_result_type_4k5k.csv", index=False)

In [ ]:
# 5k to 6k
big_df = df_5k6k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_5k6k.csv", index=False)

predicted_result_type_df_5k6k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_5k6k.to_csv("predicted_result_type_5k6k.csv", index=False)

In [ ]:
# 6k to 7k
big_df = df_6k7k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_6k7k.csv", index=False)

predicted_result_type_df_6k7k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_6k7k.to_csv("predicted_result_type_6k7k.csv", index=False)

In [ ]:
# 7k to 8k
big_df = df_7k8k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_7k8k.csv", index=False)

predicted_result_type_df_7k8k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_7k8k.to_csv("predicted_result_type_7k8k.csv", index=False)

In [ ]:
# 8k to 9k
big_df = df_8k9k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_8k9k.csv", index=False)

predicted_result_type_df_8k9k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_8k9k.to_csv("predicted_result_type_8k9k.csv", index=False)

In [ ]:
# 9k to 10k
big_df = df_9k10k
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_9k10k.csv", index=False)

predicted_result_type_df_9k10k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_9k10k.to_csv("predicted_result_type_9k10k.csv", index=False)

In [ ]:
# for 100 cases
big_df = df_100
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_ex100.csv", index=False)

predicted_df_100 = big_df
predicted_result_type_df_ex100 = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_ex100.to_csv("predicted_result_type_ex100.csv", index=False)

In [ ]:
# for 150 cases
big_df = df_150
big_df['predicted_result_type'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_df_ex150.csv", index=False)

predicted_df_50 = big_df
predicted_result_type_df_ex50 = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_ex50.to_csv("predicted_result_type_ex150.csv", index=False)

# расчёт метрик accuracy и macro F1

In [ ]:
# расчет таких метрик не актуален, потому что у нас нет колонки с истинными значениями
# можем посчитать эти метрики только для других размеченных частей (например, обучить на 50 и проверить на 100 строках)

from sklearn.metrics import accuracy_score, f1_score, classification_report

# accuracy
accuracy = accuracy_score(big_df['result_type'], big_df['predicted_result_type'])

# F1 (макроусреднение – чтобы учитывать все классы одинаково)
f1 = f1_score(big_df['result_type'], big_df['predicted_result_type'], average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1-score: {f1:.4f}")

# более подробный отчёт по каждому классу:
print(classification_report(big_df['result_type'], big_df['predicted_result_type']))

Accuracy: 0.9463
Macro F1-score: 0.6333
                       precision    recall  f1-score   support

исправительные работы       0.00      0.00      0.00         4
      лишение свободы       0.94      1.00      0.97        16
  обязательные работы       0.84      0.89      0.86        18
  ограничение свободы       0.00      0.00      0.00         2
   условное осуждение       0.95      1.00      0.97        58
                штраф       0.98      1.00      0.99        51

             accuracy                           0.95       149
            macro avg       0.62      0.65      0.63       149
         weighted avg       0.91      0.95      0.93       149



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
